In [1]:
SUF = -3
sentences = []
with open('test.txt', 'r') as f:
    for line in f:
        sentences.append(line.rstrip('\n'))

In [2]:
s_sets = [set(i.split()) for i in sentences]
s_sets_suffixes = [set([w[SUF:] for w in i.split()]) for i in sentences]
dictionary = {}
dictionary_suffixes = {}

with open('poleval_2grams.txt', 'r') as f:
    i = 0
    for line in f:
        i += 1
        if i%10000000 == 0:
            print(i)
        weight, root, subsequent = line.split()
        if int(weight) > 10:
            for s in range(len(s_sets)):
                if root in s_sets[s] and subsequent in s_sets[s]:
                    if root not in dictionary:
                        dictionary[root] = {}
                    dictionary[root][subsequent] = int(weight)
                if root[SUF:] in s_sets_suffixes[s] and subsequent[SUF:] in s_sets_suffixes[s]:
                    if root[SUF:] not in dictionary_suffixes:
                        dictionary_suffixes[root[SUF:]] = {}
                    dictionary_suffixes[root[SUF:]][subsequent[SUF:]] = int(weight)

10000000
20000000
30000000
40000000
50000000


In [3]:
with open('poleval_3grams.txt', 'r') as f:
    i = 0
    for line in f:
        i += 1
        if i%10000000 == 0:
            print(i)
        if len(line.split()) == 3:
            weight, root, subsequent1 = line.split()
            subsequent2 = None
            subsequent = subsequent1
            subsequent_s = subsequent1
        else:
            weight, root, subsequent1, subsequent2 = line.split()
            subsequent = ','.join([subsequent1,subsequent2])
            subsequent_s = ','.join([subsequent1[SUF:],subsequent2[SUF:]])
        if int(weight) > 15:
            for s in range(len(s_sets)):
                if root in s_sets[s] and subsequent1 in s_sets[s] and (subsequent2 == None or subsequent2 in s_sets[s]):
                    if root not in dictionary:
                        dictionary[root] = {}
                    dictionary[root][subsequent] = int(weight)
                if root[SUF:] in s_sets_suffixes[s] and subsequent1[SUF:] in s_sets_suffixes[s] and (subsequent2[SUF:] == None or subsequent2[SUF:] in s_sets_suffixes[s]):
                    if root[SUF:] not in dictionary_suffixes:
                        dictionary_suffixes[root[SUF:]] = {}
                    dictionary_suffixes[root[SUF:]][subsequent_s] = int(weight)

10000000
20000000
30000000
40000000
50000000
60000000
70000000
80000000
90000000
100000000
110000000
120000000
130000000
140000000
150000000
160000000
170000000


In [95]:
def rate_better(sentence):
    ans = 0
    for i in range(len(sentence) - 1):
        if sentence[i] in dictionary and sentence[i + 1] in dictionary[sentence[i]]:
            ans +=  dictionary[sentence[i]][sentence[i + 1]]
            if i < len(sentence) - 2:
                if ','.join([sentence[i + 1],sentence[i + 2]]) in dictionary[sentence[i]]:
                    ans += 2 * dictionary[sentence[i]][','.join([sentence[i + 1],sentence[i + 2]])]
        if sentence[i][SUF:] in dictionary_suffixes and sentence[i + 1][SUF:] in dictionary_suffixes[sentence[i][SUF:]]:
            ans +=  dictionary_suffixes[sentence[i][SUF:]][sentence[i + 1][SUF:]]
            if i < len(sentence) - 2:
                if ','.join([sentence[i + 1][SUF:],sentence[i + 2][SUF:]]) in dictionary_suffixes[sentence[i][SUF:]]:
                    ans += 2 * dictionary_suffixes[sentence[i][SUF:]][','.join([sentence[i + 1][SUF:],sentence[i + 2][SUF:]])]
    return ans

In [96]:
def check_pos(s, a):
    c = 0
    while s != list(a[c][1]):
        c += 1
        if c > 50000:
            print('ERROR')
            break
    return c + 1

In [99]:
import itertools
count = 0
sum_s = 0
for s in sentences:
    count += 1
    perm = list(itertools.permutations(s.split()))
    ans = [[rate_better(p), p] for p in perm]
    ans.sort(key = lambda p: p[0], reverse = True)
    sum_s += 1/check_pos(s.split(),ans)
    
print("AVERAGE: ", sum_s / count)

AVERAGE:  0.26558630794766425
